In [ ]:
from typing import *
from IPython.display import HTML, Javascript, display
import ipywidgets as wi
from PIL import Image
from io import BytesIO
from base64 import b64encode, b64decode
from pathlib import Path
from random import Random
PX = 128
root = Path("/data/natsuki/danbooru2020")
fnames = set(Path(root/"v3.txt").read_text().strip().split())
def bucket(_id: str) -> str:
    return _id[-3:].zfill(4)

In [ ]:
class Pannel(wi.VBox):
    def fname2id(self, fname: str) -> str:
        return fname.split("/")[-1].split(".")[0]
    def on_click(self, event) -> None:
        if event["new"]:
            self.bag.add(self.id)
        else:
            self.bag.discard(self.id)
        self.output.clear_output()
        with self.output:
            print(self.id, len(self.bag))
    def __init__(
        self,
        fname: str,
        bag: Set[str],
        output: wi.Output,
    ) -> None:
        self.fname = fname
        self.bag = bag
        self.output = output
        self.id = self.fname2id(fname)
        with open(fname, "rb") as f:
            buf = f.read()
        self.image = wi.Image(value=buf, format='png', width=PX, height=PX)
        self.button = wi.ToggleButton(description=self.id, layout=wi.Layout(width=f"{PX}px"), value=self.id in bag)
        self.button.observe(self.on_click, "value")
        super().__init__([self.image, self.button])

In [ ]:
class Viewer(wi.Box):
    def id2path(self, _id: str) -> str:
        return str(root/"512px"/bucket(_id)/f"{_id}.jpg")
    def __len__(self) -> int:
        return (len(self.fnames)-1)//self.N+1
    def __getitem__(self, i: int) -> List[Pannel]:
        assert i in range(len(self))
        return [
            Pannel(fname, self.bag, self.output)
            for fname in self.fnames[i*self.N:(i+1)*self.N]
        ]
        return (len(self.fnames)-1)//self.N+1
    def refresh(self) -> None:
        self.status.clear_output()
        with self.status:
            print(f"{self.i}/{len(self)}", len(self.fnames))
        self.children = tuple(self[self.i]+[self.controller])
    def on_click_p(self, event) -> None:
        self.i -= 1; self.i %= self.N; self.refresh()
    def on_click_n(self, event) -> None:
        self.i += 1; self.i %= self.N; self.refresh()
    def __init__(
        self,
        fnames: Set[str],
        N: int = 20,
        seed: int = 0,
    ) -> None:
        self.fnames = list(set(fnames))
        self.N = N
        self.seed = seed
        self.fnames.sort()
        Random(self.seed).shuffle(self.fnames)
        self.i = 0
        self.bag = set()
        self.button_p = wi.Button(description="prev", layout=wi.Layout(width=f"{PX}px"))
        self.button_p.on_click(self.on_click_p)
        self.button_n = wi.Button(description="next", layout=wi.Layout(width=f"{PX}px"))
        self.button_n.on_click(self.on_click_n)
        self.status = wi.Output(layout=wi.Layout(width=f"{PX}px"))
        self.output = wi.Output(layout=wi.Layout(width=f"{PX}px"))
        self.controller = wi.VBox([self.button_p, self.button_n, self.status, self.output])
        self.layout = layout=wi.Layout(flex_flow="row wrap")
        super().__init__()
        self.refresh()
v = Viewer(fnames)
v